# Hybrid Quantum-Classical Medical Imaging on Colab

This notebook sets up the environment to run the QML medical imaging project directly from GitHub using Google Colab.

> **Important:** Make sure you have selected **T4 GPU** runtime before running.  
> Go to **Runtime → Change runtime type → T4 GPU**


In [5]:
# 1. Setup Environment & Mount Drive
import os
import shutil
from google.colab import drive
from getpass import getpass

# Mount Drive
drive.mount('/content/drive')

# Configuration
REPO_URL = "https://github.com/DreamX55/QML-for-Medical-Imaging.git"
PROJECT_NAME = "qml_medical_imaging"  # Local folder name
PROJECT_ROOT = f"/content/{PROJECT_NAME}"

# --- CUSTOMIZE THESE PATHS IF NEEDED ---
DRIVE_DATASET_PATH = '/content/drive/MyDrive/medical_imaging_data/brain_tumor_dataset'
DRIVE_OUTPUT_PATH = '/content/drive/MyDrive/qml_medical_imaging_outputs'
# ---------------------------------------

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# 2. Verify GPU is Available
import torch
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('⚠️  No GPU detected!')
    print('Go to Runtime → Change runtime type → T4 GPU')

!nvidia-smi

PyTorch version: 2.9.0+cu128
CUDA available: True
GPU: Tesla T4
GPU Memory: 15.6 GB
Thu Feb 19 12:23:19 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |             

In [8]:
# 3. Check Dataset Existence
if not os.path.exists(DRIVE_DATASET_PATH):
    print(f"❌ Error: Dataset not found at {DRIVE_DATASET_PATH}")
    print("Please make sure you uploaded the 'brain_tumor_dataset' folder to 'medical_imaging_data' in your Google Drive.")
    # List parent dir to help debug
    parent = os.path.dirname(DRIVE_DATASET_PATH)
    if os.path.exists(parent):
        print(f"Contents of {parent}:")
        print(os.listdir(parent))
    else:
        print(f"Parent directory {parent} does not exist either.")
    raise FileNotFoundError("Dataset not found in Google Drive")
else:
    print(f"✅ Dataset found at {DRIVE_DATASET_PATH}")

✅ Dataset found at /content/drive/MyDrive/medical_imaging_data/brain_tumor_dataset


In [15]:
# 4. Clone/Update Repository
if os.path.exists(PROJECT_ROOT):
    print(f"Repo exists at {PROJECT_ROOT}. Pulling latest changes...")
    %cd {PROJECT_ROOT}
    !git pull
else:
    print(f"Cloning repo to {PROJECT_ROOT}...")
    %cd /content
    # Handle Private Repo
    try:
        !git clone {REPO_URL} {PROJECT_NAME}
    except:
        print("Public clone failed. Assuming private repo...")
        token = getpass('Enter GitHub PAT: ')
        user = REPO_URL.split('/')[-2]
        repo_name = REPO_URL.split('/')[-1]
        !git clone https://{token}@github.com/{user}/{repo_name} {PROJECT_NAME}

%cd {PROJECT_ROOT}
!pip install -r requirements.txt

# Install fast quantum simulator backend
!pip install -q pennylane-lightning

Repo exists at /content/qml_medical_imaging. Pulling latest changes...
/content/qml_medical_imaging
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 618 bytes | 618.00 KiB/s, done.
From https://github.com/DreamX55/QML-for-Medical-Imaging
   b62e989..43f55b0  main       -> origin/main
Updating b62e989..43f55b0
Fast-forward
 quantum/quantum_layer.py | 35 ++++++++++++-----------------------
 1 file changed, 12 insertions(+), 23 deletions(-)
/content/qml_medical_imaging


In [10]:
# 5. Link Data & Outputs
data_link = os.path.join(PROJECT_ROOT, 'data/brain_mri')
output_link = os.path.join(PROJECT_ROOT, 'outputs')

def force_symlink(src, dst):
    if os.path.exists(dst) or os.path.islink(dst):
        if os.path.isdir(dst) and not os.path.islink(dst):
            shutil.rmtree(dst)
        else:
            os.remove(dst)
    os.symlink(src, dst)
    print(f"Linked {src} -> {dst}")

os.makedirs(os.path.join(PROJECT_ROOT, 'data'), exist_ok=True)

# Link Data
force_symlink(DRIVE_DATASET_PATH, data_link)

# Link Outputs
if not os.path.exists(DRIVE_OUTPUT_PATH):
    os.makedirs(DRIVE_OUTPUT_PATH, exist_ok=True)
force_symlink(DRIVE_OUTPUT_PATH, output_link)

Linked /content/drive/MyDrive/medical_imaging_data/brain_tumor_dataset -> /content/qml_medical_imaging/data/brain_mri
Linked /content/drive/MyDrive/qml_medical_imaging_outputs -> /content/qml_medical_imaging/outputs


In [16]:
!python main.py --mode train \
    --data_dir data/brain_mri \
    --model_type hybrid \
    --n_qubits 10 --n_layers 2 --entanglement ring \
    --epochs 30 \
    --batch_size 32 \
    --num_workers 2 \
    --quantum_device default.qubit \
    --device cuda \
    --output_dir outputs


/content/qml_medical_imaging

TRAINING HYBRID QUANTUM-CLASSICAL MODEL
Using device: cuda

Loading data...
  Device: cuda | pin_memory: True
Loaded 6056 images from 3 classes
  - brain_glioma: 2004 images
  - brain_menin: 2004 images
  - brain_tumor: 2048 images

Data Split Statistics:
  Train: 4238 samples (70.0%)
  Val:   909 samples (15.0%)
  Test:  909 samples (15.0%)

Class Distribution by Split:
  Train: C0: 1402 (33.1%), C1: 1402 (33.1%), C2: 1434 (33.8%)
  Val: C0: 301 (33.1%), C1: 301 (33.1%), C2: 307 (33.8%)
  Test: C0: 301 (33.1%), C1: 301 (33.1%), C2: 307 (33.8%)
Classes: ['brain_glioma', 'brain_menin', 'brain_tumor']

Creating model...
Initialized PQC with 10 qubits, 2 layers
  Rotations per layer: RY + RZ (2 params per qubit)
  Final rotation layer: YES (RY + RZ before measurement)
  Total trainable parameters: 60
  Entanglement: ring
  Device: lightning.qubit
QuantumLayer initialized with 60 trainable parameters

HybridQuantumClassifier initialized:
  CNN output dim: 10
 